# Task 1

In [3]:
import pandas as pd

# load dataset
dataset = pd.read_excel("recruitment.xls")

# replace all NaN values with 0 (this is consistent with the answers in the dataset)
dataset.fillna(0, inplace=True)

### Task 1.1

In [4]:
from scipy.stats import chisquare

# perform and output the results of a chi-squared independence test between two attributes from the data
def perform_x2_test(dataset, attribute1, attribute2):
    
    N = dataset.shape[0]
    
    n1 = dataset[dataset[attribute1] == 2][attribute2].value_counts()[1]
    n2 = dataset[dataset[attribute1] == 2][attribute2].value_counts()[0]
    n3 = dataset[dataset[attribute1] == 1][attribute2].value_counts()[1]
    n4 = dataset[dataset[attribute1] == 1][attribute2].value_counts()[0]
    
    e1 = (n3 + n1) * (n1 + n2) / N
    e2 = (n4 + n2) * (n1 + n2) / N
    e3 = (n3 + n1) * (n3 + n4) / N
    e4 = (n4 + n2) * (n3 + n4) / N
    
    observed = [n1, n2, n3, n4]
    expected = [e1, e2, e3, e4]
    
    chi_sq_val, p_val = chisquare(observed, expected, ddof=2)
    
    print(attribute1 + "-" + attribute2 + " chi-square test results:\nTest statistic = " + str(chi_sq_val) + "\np-value = " + str(p_val) + "\n")

# output results of chi-squared independence test between Gender and OfferNY, and BAMEyn and OfferNY
perform_x2_test(dataset, "Gender", "OfferNY")
perform_x2_test(dataset, "BAMEyn", "OfferNY")

Gender-OfferNY chi-square test results:
Test statistic = 20.543285097740544
p-value = 5.829793346746857e-06

BAMEyn-OfferNY chi-square test results:
Test statistic = 2.718321003055136
p-value = 0.09920231543819825



### Task 1.3

In [5]:
import numpy as np

male_dataset = dataset[dataset["Gender"] == 1]
female_dataset = dataset[dataset["Gender"] == 2]

print("Percentage of males shortlisted for interview = " + str(100*np.mean(male_dataset["ShortlistedNY"])))
print("Percentage of females shortlisted for interview = " + str(100*np.mean(female_dataset["ShortlistedNY"])))
print("Percentage of males invited to interview = " + str(100*np.mean(male_dataset["Interviewed"])))
print("Percentage of females invited to interview = " + str(100*np.mean(female_dataset["Interviewed"])))
print("Percentage of males offered a job = " + str(100*np.mean(male_dataset["OfferNY"])))
print("Percentage of females offered a job = " + str(100*np.mean(female_dataset["OfferNY"])))

Percentage of males shortlisted for interview = 48.717948717948715
Percentage of females shortlisted for interview = 24.752475247524753
Percentage of males invited to interview = 34.61538461538461
Percentage of females invited to interview = 13.861386138613863
Percentage of males offered a job = 23.076923076923077
Percentage of females offered a job = 4.9504950495049505


### Task 1.4/1.5

In [6]:
perform_x2_test(dataset, "Gender", "ShortlistedNY")

Gender-ShortlistedNY chi-square test results:
Test statistic = 14.996576580734997
p-value = 0.00010770639139776792



# Task 2

### Task 2.1

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

# class implementing logistic regression
class LogReg:
    
    # initialize LogReg object, setting maximum number of iterations and initializing coefficients
    def __init__(self, max_iter=1000):
        self.max_iter = max_iter
        self.coefs = None
    
    # train the model, using gradient descent to find the optimal coefficients within the number of iterations
    def fit(self, X, y, learn_rate=0.3):
        self.coefs = np.zeros(X.shape[1] + 1, dtype=np.float32)
        N = X.shape[0]
        for t in range(self.max_iter):
            g = np.zeros(X.shape[1] + 1, dtype=np.float32) # gradient vector
            sigmoid = self.compute_sigmoid(X)
            loss = np.array(y) - sigmoid
            g[0] = np.sum(-1 * loss)
            k = 1
            for col in X.columns:
                g[k] = np.sum(-1 * loss * np.array(X[col]))
                k += 1
                
            self.coefs -= (learn_rate / N) * g # update coefficients simultaneously with respect to gradient
     
    # compute the sigmoid function for either a set of individuals, or one individual
    def compute_sigmoid(self, X):
        l = np.full(X.shape[0], self.coefs[0], dtype=np.float32)
        
        k = 1
        for col in X.columns:
            l += self.coefs[k] * np.array(X[col])
            k += 1
            
        return 1 / (1 + np.exp(-1 * l))
    
    # predict class labels for either a set of individuals, or one individual
    def predict(self, X, include_proba=False):
        sigmoid = self.compute_sigmoid(X)
        y_pred = np.where(sigmoid > 0.5, 1, 0)
        if include_proba is False:
            return y_pred
        else:            
            return y_pred, sigmoid

# columns for X variables
X_columns = ["Gender", "BAMEyn", "ShortlistedNY", "Interviewed", "FemaleONpanel"]

# get X and y
X = dataset[X_columns]
y = dataset["OfferNY"]

# perform a random 70/30 train-test split on X and y. The split will be the same each time due to random_state=10
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=10)

# create a LogReg object and fit it to the training data
lr = LogReg()
lr.fit(X_train, y_train)

### Task 2.2

In [66]:
# display accuracy and bias metrics for a given classifier and test set
def display_metrics(classifier, X_test, y_test, label):    
    y_pred = classifier.predict(X_test)
    print(label + " test set individuals = " + str(len(y_test)))
    print(label + " accuracy score = " + str(accuracy_score(y_test, y_pred)))
    print(label + " f1 score = " + str(f1_score(y_test, y_pred)))
    print(label + " precision score = " + str(precision_score(y_test, y_pred)))
    print(label + " recall score = " + str(recall_score(y_test, y_pred)))
    print(label + " % predicted 1 = " + str(100 * np.sum(y_pred) / len(y_pred)) + "\n")

# split test set into male and female test sets
X_test_male = X_test[X_test["Gender"] == 1]
y_test_male = y_test.values[np.where(X_test["Gender"] == 1)]
X_test_female = X_test[X_test["Gender"] == 2]
y_test_female = y_test.values[np.where(X_test["Gender"] == 2)]

# display accuracy metrics for males and females, and overall (on test sets)
display_metrics(lr, X_test_male, y_test_male, "Male")
display_metrics(lr, X_test_female, y_test_female, "Female")
display_metrics(lr, X_test, y_test, "Overall")

Male test set individuals = 20
Male accuracy score = 0.85
Male f1 score = 0.6666666666666665
Male precision score = 0.75
Male recall score = 0.6
Male % predicted 1 = 20.0

Female test set individuals = 64
Female accuracy score = 0.90625
Female f1 score = 0.25
Female precision score = 1.0
Female recall score = 0.14285714285714285
Female % predicted 1 = 1.5625

Overall test set individuals = 84
Overall accuracy score = 0.8928571428571429
Overall f1 score = 0.47058823529411764
Overall precision score = 0.8
Overall recall score = 0.3333333333333333
Overall % predicted 1 = 5.9523809523809526



### Task 2.3

In [74]:
# class implementing logistic regression with an adversarial component to mitigate bias. Child class of LogReg()
class AdvLogReg(LogReg):
    
    # initialize LogReg object, setting maximum number of iterations and initializing coefficients
    def __init__(self, max_iter=1000):
        LogReg.__init__(self, max_iter)
        self.coefs = None
        self.adv_coefs = None
    
    # train the model using gradient descent, also implementing an adversary model that tries to predict the sensitive attribute
    # of gender, based on the predicted label (by definition, this adversary tries to enforce demographic parity)
    def fit(self, X, y, learn_rate=0.3, weights=[]):
        # give all individuals unit weight if weights are not specified
        if len(weights) == 0:
            weights = np.ones(X.shape[0], dtype=np.int8)
        self.coefs = np.zeros(X.shape[1] + 1, dtype=np.float32)
        self.adv_coefs = np.zeros(2, dtype=np.float32)
        N = X.shape[0]
        for t in range(1, self.max_iter + 1):
            g = np.zeros(X.shape[1] + 1, dtype=np.float32) # gradient vector with respect to the loss of the predictor
            h = np.zeros(X.shape[1] + 1, dtype=np.float32) # gradient vector with respect to the loss of the adversary
            
            y_sigmoid = self.compute_sigmoid(X)
            y_loss = np.array(y) - y_sigmoid
            
            # train adversary and update its coefficients
            y_pred = np.where(y_sigmoid > 0.5, 1, 0) # predicted labels from y_sigmoid
            z_sigmoid = self.compute_adv_sigmoid(y_pred)
            z_loss = np.array(X["Gender"], dtype=np.float32) - 1 - z_sigmoid           
            self.adv_coefs[0] -= (learn_rate / N) * np.sum(-1 * z_loss)
            self.adv_coefs[1] -= (learn_rate / N) * np.sum(-1 * z_loss * y_pred)
            
            # update gradient of intercepts
            g[0] = np.sum(-1 * y_loss)
            h[0] = np.sum(-1 * z_loss)
            
            # update gradient of other coefficients
            k = 1
            for col in X.columns:
                g[k] = np.sum(-1 * y_loss * np.array(X[col]) * weights)
                h[k] = np.sum(-1 * z_loss * np.array(X[col]))
                k += 1          
            
            # hyperparameter alpha, changes over time, based on paper by Zhange
            alpha = np.sqrt(1 / t)
            
            # update coefficients as in paper by Zhange
            self.coefs -= (learn_rate / N) * (g - vector_project(g, h) - alpha * h)
            
    # compute sigmoid function for the adversary
    def compute_adv_sigmoid(self, y_pred):
        z_sigmoid = self.adv_coefs[0] + self.adv_coefs[1] * y_pred
        return 1 / (1 + np.exp(-1 * z_sigmoid))

# compute the projection of vector u onto vector v    
def vector_project(u, v):
    return (np.dot(u, v)/np.dot(v, v))*v

# get sample weights for the training set, based on the definition of demographic parity
def get_sample_weights(X_train, y_train):
    data = X_train
    data.insert(0, "OfferNY", y_train)
    N = X_train.shape[0]
    weights = list()
    for i in data.index:
        gender = data.at[i, "Gender"]
        p_gender = data["Gender"].value_counts()[gender] / N
        label = data.at[i, "OfferNY"]
        p_label = data["OfferNY"].value_counts()[label] / N
        p_exp = p_gender * p_label
        p_obs = len(data[(data["Gender"] == gender) & (data["OfferNY"] == label)]) / N
        weights.append(p_exp / p_obs)
    data.drop(columns="OfferNY", inplace=True)
    return np.array(weights, dtype=np.float32)

# compute the sample weights for the training set 
weights = get_sample_weights(X_train, y_train)

# initialize an AdvLogReg() object, and fit the training data to it, using the sample weights above
adv_lr = AdvLogReg()
adv_lr.fit(X_train, y_train, weights=weights)

C:\Users\robbi\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Task 2.4

In [72]:
display_metrics(adv_lr, X_test_male, y_test_male, "Male")
display_metrics(adv_lr, X_test_female, y_test_female, "Female")
display_metrics(adv_lr, X_test, y_test, "Overall")

Male test set individuals = 20
Male accuracy score = 0.85
Male f1 score = 0.6666666666666665
Male precision score = 0.75
Male recall score = 0.6
Male % predicted 1 = 20.0

Female test set individuals = 64
Female accuracy score = 0.9375
Female f1 score = 0.7142857142857143
Female precision score = 0.7142857142857143
Female recall score = 0.7142857142857143
Female % predicted 1 = 10.9375

Overall test set individuals = 84
Overall accuracy score = 0.9166666666666666
Overall f1 score = 0.6956521739130435
Overall precision score = 0.7272727272727273
Overall recall score = 0.6666666666666666
Overall % predicted 1 = 13.095238095238095



### Task 2.5

In [73]:
# get predictions for male and female test sets for both models
y_pred_male_reg = lr.predict(X_test_male)
y_pred_female_reg = lr.predict(X_test_female)
y_pred_male_fair = adv_lr.predict(X_test_male)
y_pred_female_fair = adv_lr.predict(X_test_female)

# calculate adverse impact ratio before fairness implementation and after
air_before = (np.sum(y_pred_female_reg) / len(y_pred_female_reg)) / (np.sum(y_pred_male_reg) / len(y_pred_male_reg))
air_after = (np.sum(y_pred_female_fair) / len(y_pred_female_fair)) / (np.sum(y_pred_male_fair) / len(y_pred_male_fair))

print("AIR before fairness implementation =", air_before)
print("AIR after fairness implementation =", air_after)

AIR before fairness implementation = 0.078125
AIR after fairness implementation = 0.546875
